In [2]:
import pandas as pd
from sklearn.model_selection import  train_test_split
import pickle
import xgboost

In [9]:
test_df = df

In [13]:
test_df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,0,22.0,1,0,7.2500,8,2
1,1,1,38.0,1,0,71.2833,2,0
2,3,1,26.0,0,0,7.9250,8,2
3,1,1,35.0,1,0,53.1000,2,2
4,3,0,35.0,0,0,8.0500,8,2


In [88]:
df = pd.read_csv("./data/titanic.csv")
df = df[["Pclass", "Sex", "Age", "SibSp", "Parch","Fare", "Cabin", "Embarked", "Survived"]]
df["Cabin"] = [x[0] if type(x) == str else "No Data" for x in df["Cabin"].values]
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Survived
0,3,male,22.0,1,0,7.2500,No Data,S,0
1,1,female,38.0,1,0,71.2833,C,C,1
2,3,female,26.0,0,0,7.9250,No Data,S,1
3,1,female,35.0,1,0,53.1000,C,S,1
4,3,male,35.0,0,0,8.0500,No Data,S,0


In [89]:
df = df.dropna().reset_index(drop=True)
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Survived
0,3,male,22.0,1,0,7.2500,No Data,S,0
1,1,female,38.0,1,0,71.2833,C,C,1
2,3,female,26.0,0,0,7.9250,No Data,S,1
3,1,female,35.0,1,0,53.1000,C,S,1
4,3,male,35.0,0,0,8.0500,No Data,S,0


In [90]:
from sklearn import preprocessing
for column in ["Sex", "Cabin", "Embarked"]:
    le = preprocessing.LabelEncoder()
    le.fit(df[column].values)
    df[column] = le.transform(df[column])

df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Survived
0,3,1,22.0,1,0,7.2500,7,2,0
1,1,0,38.0,1,0,71.2833,2,0,1
2,3,0,26.0,0,0,7.9250,7,2,1
3,1,0,35.0,1,0,53.1000,2,2,1
4,3,1,35.0,0,0,8.0500,7,2,0


In [91]:
X = df.drop("Survived", axis=1)
y = df["Survived"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [92]:
model_xgb = xgboost.XGBClassifier(n_estimators=100, max_depth=2).fit(X_train, y_train)

In [94]:
obj = [model_xgb, df]
with open("./data/obj_v2", "wb") as obj_file:
    pickle.dump(obj, obj_file)


In [3]:
with open("./data/obj", "rb") as obj:
    obj = pickle.load(obj)

[16:32:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.



In [71]:
obj[1]

,index,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Survived
0,0,3,1,22.0,1,0,7.2500,7,2,0
1,1,1,0,38.0,1,0,71.2833,2,0,1
2,2,3,0,26.0,0,0,7.9250,7,2,1
3,3,1,0,35.0,1,0,53.1000,2,2,1
4,4,3,1,35.0,0,0,8.0500,7,2,0
...,...,...,...,...,...,...,...,...,...,...
707,885,3,0,39.0,0,5,29.1250,7,1,0
708,886,2,1,27.0,0,0,13.0000,7,2,0
709,887,1,0,19.0,0,0,30.0000,1,2,1
710,889,1,1,26.0,0,0,30.0000,2,0,1


In [6]:
df = obj[1]

In [74]:
import shap

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [101]:
explainer = shap.TreeExplainer(obj[0])
shap_values = explainer.shap_values(obj[1].drop("Survived", axis=1), y=obj[1]["Survived"])
shap_values

array([[-0.47072846, -1.0599439 , -0.09178063, ...,  0.33997065,
        -0.1562994 , -0.07845347],
       [ 1.2900733 ,  2.3409467 , -0.20126575, ...,  0.569044  ,
         0.03131571,  0.22092609],
       [-1.0527108 ,  1.6280601 , -0.25417888, ...,  0.30271217,
        -0.05089022, -0.09441707],
       ...,
       [ 1.2900733 ,  2.0460784 , -0.5126162 , ...,  0.3918263 ,
         0.27856502, -0.09441707],
       [ 0.86205965, -1.1786368 , -0.2975338 , ...,  0.38965368,
         0.5846193 ,  0.36824173],
       [-0.47072846, -0.98524606,  0.6557821 , ..., -0.37606558,
        -0.14658369, -0.10315506]], dtype=float32)